In [3]:
# import necessary libraries
import pandas as pd
import sys
import os

sys.path.append("/workspaces/acs-study/data")
#goin AWS mode rn
import boto3  #interface with S3 my beloved
b3_key = os.environ['S3_READONLY_KEY']  #my secrets, not for you
b3_s = os.environ['S3_READONLY_SECRET']
print('Keys established.')

Keys established.


In [5]:
s3_client = boto3.client('s3', aws_access_key_id=b3_key, aws_secret_access_key=b3_s)  #accessing my S3 bucket
print('Connection to S3 established.')
bucket_name = 'acs-study'
object_key = 'data/chicago/crimes.csv'

csv_object = s3_client.get_object(Bucket=bucket_name, Key=object_key)
print('.csv loaded in memory')
csv_string = csv_object['Body'].read().decode('utf-8')

Connection to S3 established.
.csv loaded in memory


In [6]:
df = pd.read_csv(csv_string, engine='pyarrow')  #this takes 3+ minutes since we're dealing with >1.5 million rows

In [ ]:
df.head()